In [1]:
from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ReconstructionWrapper
from troppo.tasks.core import TaskEvaluator
from troppo.tasks.task_io import JSONTaskIO
from cobra.io import read_sbml_model, load_matlab_model
from cobra.io import write_sbml_model
from cobra.flux_analysis.variability import find_blocked_reactions
from cobamp.core import *
import cobamp as mp
from cobamp.utilities.parallel import batch_run
import pandas as pd
from json import JSONEncoder, JSONDecoder
import numpy as np
from numpy import log
import re

In [2]:
import cplex

In [3]:
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')  # find .{number} references
replace_alt_transcripts = lambda x: patt.sub('', x)  # replace .{number} with nothing

In [5]:
model=('Recon3D.xml')
data=('expression.csv')
tasks=('r3d_media_metabolites.json')
mdl=load_matlab_model('Recon3DModel_301.mat')
cmodl=[]

In [6]:
TASK_RESULTS_PATH = ('r3d_compact_task_results_tnbc_bc_new_nodrains_only_feas.json')  # task evaluation
CS_MODEL_DF_PATH = ('r3d_compact_tnbc_bc_tINIT.csv')

In [7]:
ocs = TabularReader(data, nomenclature='symbol', omics_type='transcriptomics').to_containers()

In [9]:
if cmodl==[]:
        model_consistent = read_sbml_model(model)
        model_consistent.remove_reactions(
            [r for r in model_consistent.reactions if r.id[:3] == 'DM_' or r.id[:5] == 'sink_'], remove_orphans=True)
        blocked_reactions = find_blocked_reactions(model_consistent)
        model_consistent.remove_reactions(blocked_reactions, remove_orphans=True)
        write_sbml_model(model_consistent, cmodl)  # write a model file if it doesn't exist
else:
        model_consistent = read_sbml_model(cmodl)

In [10]:
rw = ReconstructionWrapper(model_consistent, ttg_ratio=9999, gpr_gene_parse_function=replace_alt_transcripts)

In [31]:
t =0.9

In [32]:
def integration_fx(data_map):
        return [[k for k,v in data_map.get_scores().items() if (v is not None and v > t) or k in ['biomass_reaction']]]

In [33]:
def reconstruction_func(omics_container, params):
		t, rw = [params[k] for k in ['t', 'rw']]  # load parameters
		try:
			# if no errors appear, call the run_from_omics method passing the omics_container,
			# algorithm string, integration strategy (how the core is determined) and a solver
			# for fastcore, a threshold-based integration strategy retrieves core reactions if the score
			# is above the threshold t
			return rw.run_from_omics(omics_data=omics_container, algorithm='fastcore',
                                     integration_strategy=('threshold', [integration_fx]), solver='CPLEX')
		except:
			# the result from run_from_omics is a dict mapping reaction ids and a boolean flag - True if
			# the reaction is in the model or false otherwise
			# in case an error arises, assume all reactions are False
			return {r: False for r in rw.model_reader.r_ids}

In [35]:
batch_fastcore_res = batch_run(reconstruction_func, ocs, {'t': t, 'rw': rw}, threads=8)

In [36]:
fastcore_res_dict = dict(zip([('tINIT', oc.condition) for oc in ocs], batch_fastcore_res))

In [37]:
pd.DataFrame.from_dict(fastcore_res_dict, orient='index').to_csv(CS_MODEL_DF_PATH)

In [38]:
fastcore_res_dict = pd.read_csv(CS_MODEL_DF_PATH, index_col=[0, 1]).T.to_dict()

In [45]:
task_model = read_sbml_model('Recon3D.xml')

In [41]:
task_list = [t for t in JSONTaskIO().read_task(tasks)
             if len((set(t.inflow_dict) | set(t.outflow_dict)) - set([m.id for m in task_model.metabolites])) == 0]
for task in task_list:
    task.inflow_dict = {k:v if k not in task.outflow_dict.keys() else [-1000, 1000] for k,v in task.inflow_dict.items()}
    task.outflow_dict = {k:v for k,v in task.outflow_dict.items() if k not in task.inflow_dict.items()}
for task in task_list:
    task.mandatory_activity = []


In [25]:
for k in task_model.boundary:
	k.knock_out()

In [26]:
all_reactions = set([r.id for r in task_model.reactions])

In [27]:
task_eval_results = {}

In [34]:
for k, result in fastcore_res_dict.items():
    # using with statements to change the COBRA model temporarily
    # this is done to knock-out reaction not appearing the FASTCORE result
    with task_model as context_specific_model:
        protected = set([k for k, v in result.items() if v])  # get reactions included in the sample-specific model
        to_remove = all_reactions - protected  # get reactions except the protected ones
        for rid in to_remove:
            context_specific_model.reactions.get_by_id(rid).knock_out()  # knock-out reactions not in the model

        # create a task evaluator instance with the context specific model and the supplied task list and solver
        task_eval = TaskEvaluator(model=task_model, tasks=task_list, solver='CPLEX')

        # get task names (for future reference)
        task_names = task_eval.tasks

        # use the batch_function from the TaskEvaluator class (takes the name of a loaded task, a params
        # dictionary with the task evaluator associated to the 'tev' key) and set the amount of threads to be used
        batch_res_tasks = batch_run(TaskEvaluator.batch_function, task_names, {'tev': task_eval}, threads=8)
    # each element in the list of results in batch_res_tasks is a tuple of length 3 with the following:
    # 0 - boolean flag representing the task evaluation
    # 1 - Solution instance used to evaluate the task
    # 2 - A dictionary with reactions supposed to be active mapped to True/False according to that criterion

    # keep only items 0 and 2 of the task result - we don't need the flux distribution
    task_csm_res = {k: (v[0], v[2]) for k, v in dict(zip(task_names, batch_res_tasks)).items()}
    print(k,len(protected),len([v for k,v in task_csm_res.items() if v[0]]),'tasks completed.')
    # assign this dictionary to it's sample on the master results dictionary
    task_eval_results[k] = task_csm_res

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
with open(TASK_RESULTS_PATH, 'w') as f:
		f.write(JSONEncoder().encode([(k,v) for k,v in task_eval_results.items()]))